In [ ]:
import  os
import torch
import torch.nn.functional as F
import torch.utils.data as Data
from sklearn.model_selection import train_test_split
from MNN import *
import pyedflib
import numpy as np
import h5py

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix as cm
import  matplotlib.pyplot as plt

from sklearn.model_selection import KFold
import gc
from ResultCode import cm_plot,kappa
from copy import deepcopy

##迭代次数为50
EPOCH = 50
# batch size 为128
BATCH_SIZE = 128
##学习率为0.01
LEARNING_RATE = 0.01
##动量因子为0.9
MOMENTUM = 0.9
##mlp隐藏单元
mlp_hsize = 300
##rnn隐藏单元
rnn_hsize =300
row = 0
use_gpu = False
h5file = "D:/WorkSpace/Shell/datafeature/"
files = os.listdir(h5file)  # 得到文件夹下的所有文件名称
files_len = len(files)
channels = ['handfea', 'lable']
savepath = "D:/WorkSpace/Shell/MNN/result_new20/label/"

# 保存  训练测试结果
def saveResult(testresult, loss_train, loss_test, acc, koldi, mnn):
    np.savetxt('./result_new20/test_matrix/testresult' + koldi + '.txt', testresult)
    np.savetxt('./result_new20/loss/loss_train' + koldi + '.txt', loss_train)
    np.savetxt('./result_new20/acc/acc_' + koldi + '.txt', acc)
    np.savetxt('./result_new20/loss/loss_test' + koldi + '.txt', loss_test)
    torch.save(mnn, './net/net' + koldi + '.pkl')  # entire net


# 训练MNN
def mnnTrain(train_x, train_y, loss_all, mnn, loss_func,optimizer):
    train_y_shape = train_y.shape[0]
    train_y = train_y.view(train_y.shape[0]).type(torch.LongTensor)

    if (use_gpu):
        train_x, train_y = train_x.cuda(), train_y.cuda()
    # mnn output
    output = mnn(train_x.type(torch.FloatTensor))
    loss = loss_func(output, train_y)

    # cross entropy loss
    loss_all = loss_all + loss
    optimizer.zero_grad()  # clear gradients for this training step
    loss.backward()  # backpropagation, compute gradients
    optimizer.step()  # apply gradients


# 画图  损失  准确率
def plot_acc_loss(x1, x2, train_acc, loss_train, loss_test, foidi):
    plt.plot(x1, train_acc, 'b.-', label='Train accuracy')
    plt.title('accuracy vs epoches')
    plt.xlabel('epoches')
    plt.ylabel('accuracy')

    plt.savefig("./result_new20/picture/accuracy" + foidi + ".svg")
    plt.show()


    plt.plot(x2, loss_train, 'r.-', label=u'loss_train')
    plt.title('loss vs epoches')
    plt.xlabel('epoches')
    plt.ylabel('loss')
    plt.savefig("./result_new20/picture/train_loss" + foidi + ".svg")
    plt.show()
    plt.xlabel('epoches')
    plt.plot(x2, loss_test, 'b.-', label=u'loss_test')
    plt.title('loss vs epoches')
    plt.ylabel('loss')
    plt.savefig("./result_new20/picture/test_loss" + foidi + ".svg")
    plt.show()


def del_collect(del_var):

    del del_var
    gc.collect()


def setDataset(h5file, train_index, type=0):
    global eeg_data, labels
    flag = 0
    ##训练集
    for filei in train_index:
        with h5py.File(h5file + files[filei], 'r') as fileh5:
            #    print(fileh5[channels[5]].value.shape)
            if flag == 0:

                eeg_data = torch.from_numpy(fileh5[channels[0]].value)
                labels = torch.from_numpy(fileh5[channels[1]].value)
                flag = 1
            else:
                eeg_data = torch.cat((eeg_data, torch.from_numpy(fileh5[channels[0]].value)),dim=0)
                labels = torch.cat((labels, torch.from_numpy(fileh5[channels[1]].value)), dim=0)
    if type == 1:
        torch_dataset = Data.TensorDataset(eeg_data, labels.squeeze())
        train_loader = Data.DataLoader(dataset=torch_dataset, batch_size=BATCH_SIZE, shuffle=False,num_workers=1)  ##num_workers是加载文件的核心数
        return train_loader
    else:
        return eeg_data, labels.squeeze()

##保存结果报告文件
def save_to_file(file_name, contents):
    fh = open(file_name, 'w')
    fh.write(contents)
    fh.close()


def savePred(savepath, filename, label, pred):

    with h5py.File(savepath + filename, 'w') as fileh5:
        fileh5['label'] =label
        fileh5['pred'] =label


if __name__ =='__main__':


    print("开始")

    for index in range(1, 21):
        train_index, test_index = train_test_split(range(files_len), test_size=0.2, random_state=index)
        save_list = []
        save_list.append(train_index)
        save_list.append(test_index)
        save_to_file("D:/WorkSpace/Shell/MNN/list/list_"+ str(index) +".txt", str(save_list))
        mnn = MNNs()
        optimizer = torch.optim.SGD(mnn.parameters(), lr=LEARNING_RATE)
        loss_func = nn.CrossEntropyLoss()
        if (use_gpu):
            mnn = mnn.cuda()
            loss_func = loss_func.cuda()

        # 定义数组
        Loss_list_train = []
        Loss_list_test = []
        Accuracy_list = []
        Kappa = []
        Loss_list2 = []
        Accuracy_list2 = []
        Matrix_all = []
        global save_test_pred_all
        global save_test_label_all

        # 我这里迭代了80次，所以x的取值范围为(0，80)，然后再将每次相对应的准确率以及损失率附在x上
        x1 = range(0, EPOCH)
        x2 = range(0, EPOCH)
        y1 = Accuracy_list
        y2 = Loss_list_train
        y3 = Loss_list_test
        best_acc = 0

        print("开始训练------")
        # 训练MNN网络

        max_accuracy = 0
        global save_test_pred_all
        for epoch in range(EPOCH):
            loss_all = 0
            loss_test_all = 0
            acc_train_all = 0
            step_all = 0
            test_step_all = 0

            print("开始训练模型---------")
            train_part_len = len(train_index)//20 ##训练集划分長度
            test_part_len = len(test_index)//8

            train_part_len = len(train_index) // train_part_len
            test_part_len = len(test_index) // test_part_len
            for train_part in range(train_part_len):
                if train_part == 0:
                    train_part_index = train_index[train_part_len * train_part:train_part_len * (train_part + 1)]
                elif train_part == (train_part_len-1):
                    train_part_index = train_index[train_part_len * train_part:len(train_index)]
                else:
                    train_part_index = train_index[train_part_len * train_part:train_part_len * (train_part + 1)]
                for step, (train_x, train_y) in enumerate(setDataset(h5file, train_part_index, type=1)):  # 设定训练数据

                    train_y_shape = train_y.shape[0]
                    train_y = train_y.view(train_y.shape[0])

                    if (use_gpu):
                        train_x, train_y = train_x.cuda(), train_y.cuda()
                    # mnn output
                    output = mnn(train_x.type(torch.FloatTensor))
                    loss_train = loss_func(output, train_y.type(torch.LongTensor))


                    # cross entropy loss

                    optimizer.zero_grad()  # clear gradients for this training step
                    loss_train.backward()  # backpropagation, compute gradients
                    optimizer.step()  # apply gradients

            print("训练模型完毕")
            test_step_all = 0
            ##测试集切割

            for test_part in range(test_part_len):
                if test_part == 0:
                    test_part_index = test_index[test_part_len * test_part:test_part_len * (test_part + 1)]
                elif test_part == test_part_len-1:
                    test_part_index = test_index[test_part_len * test_part:len(test_index)]
                else:
                    test_part_index = test_index[test_part_len * test_part:test_part_len * (test_part + 1)]

                for step, (test_x, test_y) in enumerate(setDataset(h5file, test_part_index, type=1)):  # 设定训练数据

                    if step == 0:

                        test_label_all = deepcopy(test_y)
                    else:
                        test_label_all = torch.cat((test_label_all, test_y), dim=0)
                    test_y = test_y.view(test_y.shape[0])

                    # mnn output
                    output = mnn(test_x.type(torch.FloatTensor))
                    test_pred_y = torch.max(F.softmax(output), 1)[1]


                    if step == 0:
                        test_pred_all = deepcopy(test_pred_y)
                    else:
                        test_pred_all = torch.cat((test_pred_all, test_pred_y), dim=0)

                    loss_test = loss_func(output, test_y.type(torch.LongTensor))
                    # cross entropy loss
                    loss_test_all = loss_test_all + loss_test
                test_step_all = step + 1 + test_step_all

            # #测试集

            test_accuracy = accuracy_score(test_label_all.numpy().squeeze(), test_pred_all.numpy().squeeze())
            loss_test_avg = loss_test_all / test_step_all
            Loss_list_test.append(loss_test_avg)


            Accuracy_list.append(test_accuracy)
            # accuracy = sum(test_labels1 == pred_y)/ test_labels.numpy().squeeze().size
            print(str(index) + '_Epoch: ', epoch + 1,
                '| test loss: %.4f' % loss_test_avg, '| test accuracy: %.4f' % test_accuracy)

            gc.collect()
            torch.cuda.empty_cache()
            Matrix_all.append(cm(np.array(test_label_all).reshape(-1).squeeze(), np.array(test_pred_all).reshape(-1).squeeze()))
            Kappa.append(kappa(cm(np.array(test_label_all).reshape(-1).squeeze(), np.array(test_pred_all).reshape(-1).squeeze())))

            if epoch!=50:

                del test_label_all,test_pred_all
                gc.collect()
                torch.cuda.empty_cache()
            else:
                save_test_pred_all = test_label_all
                save_test_label_all = test_pred_all
                del test_label_all,test_pred_all
                gc.collect()
                torch.cuda.empty_cache()

        # #混淆矩阵

        print("结束训练网络-------")

        for index in test_index:
            test_x, test_y = setDataset(h5file, [index], type=0)
            # mnn output
            output = mnn(test_x.type(torch.FloatTensor))
            test_pred_y = torch.max(F.softmax(output), 1)[1]
            savePred(savepath, files, test_y, test_pred_y)


        result1 = np.array(Matrix_all).reshape(-1, 25)
        # result4 = np.array(Loss_list_test)
        result2 = np.array(Loss_list_train)
        result3 = np.array(Loss_list_test)
        result4 = np.array(Accuracy_list)
        saveResult(result1, result2, result3, result4, "_" + str(index).zfill(2), mnn)
        # 画图及保存
        plot_acc_loss(x1, x2, y1, y2, y3, "_" + str(index).zfill(2))
        result5 = np.array(Kappa)
        cm_plot(np.array(save_test_pred_all).reshape(-1).squeeze(), np.array(save_test_label_all).reshape(-1).squeeze(),
                "_" + str(index))
        np.savetxt('./result_new20/kappa/kappa_' + str(index) + '.txt', result5)
        if 'save_test_pred_all' in vars():
            del save_test_pred_all,save_test_label_all
            gc.collect()
        torch.cuda.empty_cache()
        gc.collect()
    print("结束")





开始
开始训练------
开始训练模型---------
训练模型完毕
1_Epoch:  1 | test loss: 1.3832 | test accuracy: 0.5228
开始训练模型---------
训练模型完毕
1_Epoch:  2 | test loss: 0.7394 | test accuracy: 0.7632
开始训练模型---------
训练模型完毕
1_Epoch:  3 | test loss: 0.6138 | test accuracy: 0.8009
开始训练模型---------
训练模型完毕
1_Epoch:  4 | test loss: 0.6249 | test accuracy: 0.8035
开始训练模型---------
训练模型完毕
1_Epoch:  5 | test loss: 0.6196 | test accuracy: 0.8026
开始训练模型---------
训练模型完毕
1_Epoch:  6 | test loss: 0.5882 | test accuracy: 0.7947
开始训练模型---------
训练模型完毕
1_Epoch:  7 | test loss: 0.5694 | test accuracy: 0.8140
开始训练模型---------


D:\Software\Anaconda\envs\pytorch_gpu\lib\site-packages\ipykernel_launcher.py:107: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
D:\Software\Anaconda\envs\pytorch_gpu\lib\site-packages\ipykernel_launcher.py:108: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
D:\Software\Anaconda\envs\pytorch_gpu\lib\site-packages\ipykernel_launcher.py:111: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
D:\Software\Anaconda\envs\pytorch_gpu\lib\site-packages\ipykernel_launcher.py:112: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
D:\Software\Anaconda\envs\pytorch_gpu\lib\site-packages\ipykernel_launcher.py:237: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
